# Reconstruction in ROOT

In [1]:
import os
import sys
import ROOT
import time

sys.path.append("..")
from helpers import filename_for

Welcome to JupyROOT 6.16/00


In [2]:
def reconstruction(distance, doubleplane, energy, erel, neutron, physics, subrun, overwrite=True):
    inpfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "digi.root")
    simfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "simu.root")
    parfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "para.root")
    outfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "test.root")

    if not os.path.isfile(inpfile):
        print(f"Input {inpfile} does not exist")
        return

    if os.path.isfile(outfile):
        if overwrite:
            os.remove(outfile)
        else:
            print(f"Output {outfile} exists and overwriting is disabled")
            return

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")

    # I/O
    run = ROOT.FairRunAna()
    ffs = ROOT.FairFileSource(os.fspath(inpfile))
    ffs.AddFriend(os.fspath(simfile))
    run.SetSource(ffs)
    run.SetSink(ROOT.FairRootFileSink(os.fspath(outfile)))

    # Connect Runtime Database and Calorimetric Par file
    rtdb = run.GetRuntimeDb()
    pario = ROOT.FairParRootFileIo(False)
    pario.open(os.fspath(parfile))
    rtdb.setFirstInput(pario)
    rtdb.setOutput(pario)
    rtdb.saveOutput()

    # Cheating Muliplicity
    run.AddTask(ROOT.R3BNeulandMultiplicityCheat("NeulandPrimaryHits", "NeulandMultiplicityCheat"))
    
    # Perfect Reco
    run.AddTask(ROOT.R3BNeulandNeutronsCheat("NeulandMultiplicityCheat", "NeulandPrimaryHits", "NeulandNeutronsCheat"))
    run.AddTask(ROOT.R3BNeulandNeutronReconstructionMon("NeulandNeutronsCheat", "NeulandNeutronReconstructionMonCheat"))

    # RValue
    run.AddTask(ROOT.R3BNeulandNeutronsRValue(energy, "NeulandMultiplicityCheat", "NeulandClusters", "NeulandNeutronsRValue"))
    run.AddTask(ROOT.R3BNeulandNeutronReconstructionMon("NeulandNeutronsRValue", "NeulandNeutronReconstructionMonRValue"))
       
    # Scikit
    run.AddTask(
        ROOT.R3BNeulandNeutronsScikit(
            "models/15m_30dp_600AMeV_500keV_4n_AdaBoostClassifier.pkl", #"models/15m_30dp_600AMeV_500keV_4n_RandomForestClassifier.pkl",
            "NeulandMultiplicityCheat",
            "NeulandClusters",
            "NeulandNeutronsScikit",
        )
    )
    run.AddTask(ROOT.R3BNeulandNeutronReconstructionMon("NeulandNeutronsScikit", "NeulandNeutronReconstructionMonScikit"))
   
    
    run.Init()
    run.Run(0, 0)
    run.TerminateRun()

In [3]:
start = time.time()
reconstruction(15, 30, 600, 500, 4, "inclxx", 1)
stop = time.time()
print(stop - start)

R3BNeulandNeutronsScikit running TPython with Python version 3.7.6 (default, Mar 10 2020, 10:53:38) [GCC 8.3.1 20190311 (Red Hat 8.3.1-3)]
115.27367043495178

************************************************************* 
     initialisation for run id 1587569958
************************************************************* 
-I- FairRunTimeDB::InitContainer() FairGeoParSet
Container FairGeoParSet initialized from ROOT file.
-I- FairRunTimeDB::InitContainer() FairBaseParSet
Container FairBaseParSet initialized from ROOT file.

************************************************************* 
     initialisation for run id 1587569958
************************************************************* 
-I- FairRunTimeDB::InitContainer() FairGeoParSet
-I- FairRunTimeDB::InitContainer() FairBaseParSet
-I- FairRunTimeDB::InitContainer() R3BFieldPar
[ERROR] init() R3BFieldPar not initialized
New field at 0, type -1
[WARN] FairRootManager::GetOutFile() deprecated. Use separate file to store additional 

Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 3000
Info in <TGeoManager::CloseGeometry>: 12004 nodes/ 8 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Error in <FairRuntimeDb::initContainers()>: Error occured during initialization
-W- FairRunAna::GetField: Unknown field type -1
Warning in <TTree::Bronch>: FairEventHeader cannot be split, resetting splitlevel to 0
Error in <TFile::mkdir>: An object with name NeulandNeutronReconstructionMonCheat exists already
Error in <TFile::mkdir>: An object with name NeulandNeutronReconstructionMonRValue exists already
Error in <TFile::mkdir>: An object with name NeulandNeutronReconstructionMonScikit

In [4]:
%%javascript
var s = document.createElement('script');
s.src='https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js';
document.body.appendChild(s);

<IPython.core.display.Javascript object>

In [5]:
%jsroot on
file = filename_for(15, 30, 600, 500, 4, "inclxx", 1, "test.root")
tfile = ROOT.TFile.Open(os.fspath(file))

c1 = ROOT.TCanvas("c1", "c1", 800, 600)

h1 = tfile.Get("NeulandNeutronReconstructionMonCheat/fhErel4").Clone()
print(h1.GetEntries())
h1.Rebin(10)
h1.SetLineColor(1)

h2 = tfile.Get("NeulandNeutronReconstructionMonScikit/fhErel4").Clone()
print(h2.GetEntries())
h2.Rebin(10)
h2.SetLineColor(2)

h3 = tfile.Get("NeulandNeutronReconstructionMonRValue/fhErel4").Clone()
print(h3.GetEntries())
h3.Rebin(10)
h3.SetLineColor(3)

h1.Draw("")
h2.Draw("same")
h3.Draw("same")

c1.Draw()

8238.0
8238.0
8238.0
